In [124]:
using Pkg
Pkg.activate("CT")
using CT
include("global_adder_passthrough.jl")

# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO, add1, power_mpo
using Random


  Activating project at `~/CT_MPS_mini/CT`


In [125]:
# Generate list of right translations of 1:L more efficiently
# Using array comprehension and avoiding modulo operations where possible
translations = [circshift(1:L, shift) for shift in 0:L-1]

function fold(translation, ancilla::Int)
    if ancilla ==0
        ram_phy = [i for pairs in zip(translation[1:(L÷2)], reverse(translation[(L÷2+1):L])) for i in pairs]
    elseif ancilla ==1
        ram_phy = vcat(L+1,[i for pairs in zip(translations[1:(L÷2)], reverse(translations[(L÷2+1):L])) for i in pairs])
    elseif ancilla ==2
        error("Not implemented yet")
    end

    # phy_ram[physical] = actual in ram
    phy_ram = fill(0, L+ancilla)
    for (ram, phy) in enumerate(ram_phy)
        phy_ram[phy] = ram
    end

    return ram_phy, phy_ram
end

conn_pairs = Dict()
for (i1, translation) in enumerate(translations)
    # println(fold(translation, 0))
    ram_phy, phy_ram = fold(translation, 0)
    # println(ram_phy)
    pairs = []
    for i in reverse(collect(1:L)[2:end])
        current_pos = findfirst(x -> x == i, ram_phy)
        next_pos = findfirst(x -> x == (i-1)%L, ram_phy)
        pair = (current_pos, next_pos)
        push!(pairs, pair)
        # println(i, " is in position $currest_pos carry into $(i-1) which is in position $next_pos")
    end

    conn_pairs[i1] = pairs
end


In [126]:
L = 8
i1 = 1
ancilla = 0
folded = true
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
pairing = conn_pairs[i1]
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)

(Index{Int64}[(dim=2|id=86|"Qubit,Site,n=1"), (dim=2|id=902|"Qubit,Site,n=2"), (dim=2|id=319|"Qubit,Site,n=3"), (dim=2|id=250|"Qubit,Site,n=4"), (dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=558|"Qubit,Site,n=8")], [1, 8, 2, 7, 3, 6, 4, 5], [1, 3, 5, 7, 8, 6, 4, 2], [1, 2, 3, 4, 5, 6, 7, 8])

In [127]:
pairing
# each tuple in pairing (x,y) means that the carry out leg from T_tensor in ram position x is connected to the carry in leg of T_tensor in ram position y


7-element Vector{Any}:
 (2, 4)
 (4, 6)
 (6, 8)
 (8, 7)
 (7, 5)
 (5, 3)
 (3, 1)

In [128]:
# need to initialize the gate_vec's according to the pairing first
carry_links = [Index(2, "Carry,c=$(ram_pos-1)") for ram_pos in 1:L+1]
gate_vec = []
for pair in pairing 
    start_pos = pair[1]
    end_pos = pair[2]
    if end_pos - start_pos == 1
        # forward one 
        T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[start_pos], qubit_site[start_pos], prime(qubit_site[start_pos]), carry_links[start_pos], carry_links[end_pos])
        push!(gate_vec, T_tensor)
        continue
    elseif end_pos - start_pos == 2
        mid_pos = (start_pos + end_pos) ÷ 2
        # next nearest neighbor case, need to create two qubit gates
        T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[start_pos], qubit_site[start_pos], prime(qubit_site[start_pos]), carry_links[start_pos], carry_links[mid_pos])
        id_tensor = create_identity_tensor_4d(qubit_site[mid_pos], prime(qubit_site[mid_pos]), carry_links[mid_pos], carry_links[end_pos])
        gate = T_tensor * id_tensor
        push!(gate_vec, gate)
    elseif end_pos - start_pos == -2
        mid_pos = (start_pos + end_pos) ÷ 2
        # next nearest neighbor case, need to create two qubit gates
        T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[start_pos], qubit_site[start_pos], prime(qubit_site[start_pos]), carry_links[start_pos], carry_links[mid_pos])
        id_tensor = create_identity_tensor_4d(qubit_site[mid_pos], prime(qubit_site[mid_pos]), carry_links[mid_pos], carry_links[end_pos])
        gate = T_tensor * id_tensor
        push!(gate_vec, gate)
    elseif end_pos - start_pos == -1
        # backward one
        T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[start_pos], qubit_site[start_pos], prime(qubit_site[start_pos]), carry_links[start_pos], carry_links[end_pos])
        push!(gate_vec, T_tensor)
        continue
    else
        # more than next nearest neighbor, not implemented yet
        error("Not implemented yet")
    end
end

# msb T tensor also needs to be included
lsb = pairing[begin][1];
msb_pos = pairing[end][2];
msb_plus_one = (pairing[end][1] + pairing[end][2]) ÷ 2;
msb_T_tensor = create_addition_tensor_with_carry(shift_1_3_bits[msb_pos], qubit_site[msb_pos], prime(qubit_site[msb_pos]), carry_links[msb_pos], carry_links[msb_plus_one]);
push!(gate_vec, msb_T_tensor);

In [129]:
# initialize the boundary condition tensor
lsb_tensor = onehot(carry_links[2]=>1)
# discard the carry out of the msb tensor
msb_tensor = ITensor(carry_links[1])
msb_tensor[carry_links[1]=>1] = 1.0
msb_tensor[carry_links[1]=>2] = 1.0

1.0

Actually i think maybe i dont need the id_tensor_4d at all; i can direct contract them according to the pairing sequence

In [ ]:
# fixing the boundary condition tensors
for igate in 1:length(gate_vec)
    gate = gate_vec[igate]
    if carry_links[2] in inds(gate)
        gate_vec[igate] = gate * msb_tensor
    end
    if carry_links[1] in inds(gate)
        gate_vec[igate] = gate * lsb_tensor
    end
end


In [131]:
for gate in gate_vec
    println(inds(gate))
end


((dim=2|id=902|"Qubit,Site,n=2"), (dim=2|id=902|"Qubit,Site,n=2")', (dim=2|id=2|"Carry,c=1"), (dim=2|id=78|"Carry,c=3"), (dim=2|id=319|"Qubit,Site,n=3"), (dim=2|id=319|"Qubit,Site,n=3")', (dim=2|id=959|"Carry,c=0"))
((dim=2|id=250|"Qubit,Site,n=4"), (dim=2|id=250|"Qubit,Site,n=4")', (dim=2|id=78|"Carry,c=3"), (dim=2|id=571|"Carry,c=5"), (dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=204|"Qubit,Site,n=5")')
((dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=376|"Qubit,Site,n=6")', (dim=2|id=571|"Carry,c=5"), (dim=2|id=224|"Carry,c=7"), (dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=198|"Qubit,Site,n=7")')
((dim=2|id=558|"Qubit,Site,n=8"), (dim=2|id=558|"Qubit,Site,n=8")', (dim=2|id=224|"Carry,c=7"), (dim=2|id=295|"Carry,c=6"))
((dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=198|"Qubit,Site,n=7")', (dim=2|id=295|"Carry,c=6"), (dim=2|id=695|"Carry,c=4"), (dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=376|"Qubit,Site,n=6")')
((dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=204|"Qubit,Site,n=5")', (dim=2|id=695|"Carry

In [132]:
for idx in 1:length(gate_vec)-1
    println(inds(gate_vec[idx] * gate_vec[idx+1]))
end

((dim=2|id=902|"Qubit,Site,n=2"), (dim=2|id=902|"Qubit,Site,n=2")', (dim=2|id=2|"Carry,c=1"), (dim=2|id=319|"Qubit,Site,n=3"), (dim=2|id=319|"Qubit,Site,n=3")', (dim=2|id=959|"Carry,c=0"), (dim=2|id=250|"Qubit,Site,n=4"), (dim=2|id=250|"Qubit,Site,n=4")', (dim=2|id=571|"Carry,c=5"), (dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=204|"Qubit,Site,n=5")')
((dim=2|id=250|"Qubit,Site,n=4"), (dim=2|id=250|"Qubit,Site,n=4")', (dim=2|id=78|"Carry,c=3"), (dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=204|"Qubit,Site,n=5")', (dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=376|"Qubit,Site,n=6")', (dim=2|id=224|"Carry,c=7"), (dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=198|"Qubit,Site,n=7")')
((dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=376|"Qubit,Site,n=6")', (dim=2|id=571|"Carry,c=5"), (dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=198|"Qubit,Site,n=7")', (dim=2|id=558|"Qubit,Site,n=8"), (dim=2|id=558|"Qubit,Site,n=8")', (dim=2|id=295|"Carry,c=6"))
((dim=2|id=558|"Qubit,Site,n=8"), (dim=2|id=558|"Qubit,Site,n=8")',

In [133]:
length(gate_vec)

8

In [134]:
for igate in 1:length(gate_vec)
    gate = gate_vec[igate]
    pair = pairing[igate]
    println(pair[1], "-->", pair[2])
    println(inds(gate))
end
# for idx in 1:length(gate_vec), find all the gates that has qubit_site[idx] as an index, and put them in a list

2-->4
((dim=2|id=902|"Qubit,Site,n=2"), (dim=2|id=902|"Qubit,Site,n=2")', (dim=2|id=2|"Carry,c=1"), (dim=2|id=78|"Carry,c=3"), (dim=2|id=319|"Qubit,Site,n=3"), (dim=2|id=319|"Qubit,Site,n=3")', (dim=2|id=959|"Carry,c=0"))
4-->6
((dim=2|id=250|"Qubit,Site,n=4"), (dim=2|id=250|"Qubit,Site,n=4")', (dim=2|id=78|"Carry,c=3"), (dim=2|id=571|"Carry,c=5"), (dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=204|"Qubit,Site,n=5")')
6-->8
((dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=376|"Qubit,Site,n=6")', (dim=2|id=571|"Carry,c=5"), (dim=2|id=224|"Carry,c=7"), (dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=198|"Qubit,Site,n=7")')
8-->7
((dim=2|id=558|"Qubit,Site,n=8"), (dim=2|id=558|"Qubit,Site,n=8")', (dim=2|id=224|"Carry,c=7"), (dim=2|id=295|"Carry,c=6"))
7-->5
((dim=2|id=198|"Qubit,Site,n=7"), (dim=2|id=198|"Qubit,Site,n=7")', (dim=2|id=295|"Carry,c=6"), (dim=2|id=695|"Carry,c=4"), (dim=2|id=376|"Qubit,Site,n=6"), (dim=2|id=376|"Qubit,Site,n=6")')
5-->3
((dim=2|id=204|"Qubit,Site,n=5"), (dim=2|id=204|"Qub

LoadError: BoundsError: attempt to access 7-element Vector{Any} at index [8]

In [ ]:
# for idx in 1:length(gate_vec), find all the gates that has qubit_site[idx] as an index, and put them in a list
# then contract all the gates in the list together
# then svd according to the qubit_site index 

In [ ]:
# Example: find intersection between two sets a and b
a = Set([1, 2, 3, 4])
b = Set([3, 4, 5, 6])
intersection = intersect(a, b)
println("Intersection: ", intersection)


Intersection: Set([4, 3])


In [ ]:
intersection = intersect(Set(inds(gate_vec[1])), Set(inds(gate_vec[2])))

Set{Index{Int64}} with 1 element:
  (dim=2|id=497|"Carry,c=3")

In [ ]:
commoninds(gate_vec[1], gate_vec[2])

1-element Vector{Index{Int64}}:
 (dim=2|id=497|"Carry,c=3")